# Data Reduction and Time Series Analysis Pipeline

## Import Packages

In [ ]:
import sys
import os
from os import chdir 
import numpy as np
import pandas as pd
import astroalign as aa
import sep
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy.stats import sigma_clip, SigmaClip
from astropy import units as u 
from astropy.timeseries import TimeSeries
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import LombScargle
from PyAstronomy.pyasl import foldAt

# from platform import python_version
# python_version()

# Import Data

In [ ]:
#change present working directory (pwd)
os.chdir("/Volumes/KT/data/")

#import fringing template
image_file = get_pkg_data_filename('fringe_new_I.fits')

#establish variables to call on certain directorys within data file
bias_dir = os.getcwd() + '/2017_12_21/bias/'
flat_dir = os.getcwd() + '/2017_12_20/flat/'
science_dir = os.getcwd() + '/2017_12_21/science/LP349/'

#Select which files you want to import by prefix
flat_list = [filename for filename in os.listdir(flat_dir) if filename.startswith("Flat_I")]
bias_list = [filename for filename in os.listdir(bias_dir) if filename.startswith("master")]
science_list = os.listdir(science_dir)
science_list = [filename for filename in os.listdir(science_dir) if filename.startswith("LP349")]

#open each fits file and check data
#flats
for flat in flat_list:
    this_flat = flat_dir + flat
    hdul = fits.open(this_flat)
    #print (this_flat)
    print (flat + '\t' + 'First frame: ' + hdul[0].header['FRAME'] + '\t' + 'Exposure (sec):' + str(hdul[0].header['EXPOSURE']) + '\t' + 'Number of integrations:' + str(hdul[0].header['NUMKIN'])) 
print("---------------------------------------------------------------------------------------------------------------")
#bias
for bias in bias_list:
    this_bias = bias_dir + bias
    hdul = fits.open(this_bias)
    print (bias + '\t' + 'First frame: ' + hdul[0].header['FRAME'] + '\t' + 'Exposure (sec):' + str(hdul[0].header['EXPOSURE']) + '\t' + 'Number of integrations:' + str(hdul[0].header['NUMKIN']))
print("---------------------------------------------------------------------------------------------------------------")
#science
for science in science_list:
    this_science = science_dir + science
    hdul = fits.open(this_science, ignore_missing_end=True)
    #print (hdul[0].header)
    print (science + '\t' + 'First frame: ' + hdul[0].header['FRAME'] + '\t' + hdul[0].header['DATE'] + '\t' + 'Exposure (sec):' + str(hdul[0].header['EXPOSURE']) + '\t' + 'Number of integrations:' + str(hdul[0].header['NUMKIN']))     
    

## Initialize Master Bias

In [ ]:
#set the bias path
bias_file = 'masterbias_11.fits'
biaspath = bias_dir + bias_file
print(biaspath)

#function to remove bias
def bias_subtract(filepath,biaspath):
    bias_data = fits.getdata(get_pkg_data_filename(biaspath),ext=0)
    image_stack_raw = fits.getdata(get_pkg_data_filename(filepath),ext=0)
    image_stack_biased = image_stack_raw - bias_data
    return(image_stack_biased)

## initialize Master Flat

In [ ]:
#view flats

In [ ]:
#determine best flat
#remove bias and print average
flatpath = flat_dir + 'Flat_I__005.fits'
this_flat_stack = bias_subtract(flatpath,biaspath)
master_flat05 = np.average(this_flat_stack, axis=0)

# Flat_I__006.fits
#remove bias and print average
flatpath = flat_dir + 'Flat_I__006.fits'
this_flat_stack = bias_subtract(flatpath,biaspath)
master_flat06 = np.average(this_flat_stack, axis=0)
plt.figure(figsize=(4,4))

#divide by median to remove systemic dieviation, each variable now has median of 1
#normalize flats
flat05 = master_flat05/np.median(master_flat05)
flat06 = master_flat06/np.median(master_flat06)
#print(np.median(test05))
#print(np.median(test06))

#subtract the diff
flatdiff = flat05 - flat06
#Divide the flats
flatdiv = flat05/flat06


#plot flats 
plt.figure(figsize = (22,6))
plt.subplot(1, 2, 1)
plt.title('Flat 5 I band')
plt.imshow(master_flat05,  origin='lower', cmap='gray')
plt.colorbar();

plt.subplot(1, 2, 2)
plt.title('Flat 6 I band')
plt.imshow(master_flat06,  origin='lower', cmap='gray')
plt.colorbar();

#plot normalized flats 
plt.figure(figsize = (22,6))
plt.subplot(1, 2, 1)
plt.imshow(flat05,  origin='lower', cmap='gray')
plt.title('Normalized Flat 5 I band')
plt.colorbar();

plt.subplot(1, 2, 2)
plt.imshow(flat06,  origin='lower', cmap='gray')
plt.title('Normalized Flat 6 I band')
plt.colorbar();

#plot difference in flats
plt.figure(figsize = (22,6))
plt.subplot(1, 2, 1)
plt.imshow(flatdiff,  origin='lower', cmap='gray')
plt.title('Flat 5 - Flat 6 I band')
plt.colorbar();

plt.subplot(1, 2, 2)
plt.imshow(flatdiv,  origin='lower', cmap='gray')
plt.title('Flat 5 / Flat 6 I band')
plt.colorbar();


In [ ]:
#visualize frames vs counts for flat stack
def MedianCountValueTS(flat_FITSfile):
    frames = []
    counts = []
    count = 1
    flatpath = flat_dir + flat_FITSfile
    flat_stack = bias_subtract(flatpath,biaspath)
    for frame in flat_stack:
        counts.append(np.median(frame))
        frames.append(count)
        count = count + 1
    return [[frames], [counts]];

flat5 = MedianCountValueTS('Flat_I__005.fits')
flat6 = MedianCountValueTS('Flat_I__006.fits')

#plot frames vs counts 
plt.figure(figsize = (22,6))
plt.subplot(1, 2, 1)
plt.plot(flat5[0], flat5[1], 'ko-')
plt.title('Flat__5')
plt.ylabel('Median counts/frame')
plt.xlabel('Frame #')
plt.subplot(1, 2, 2)
plt.title('Flat__6')
plt.plot(flat6[0], flat6[1], 'r.-')
plt.xlabel('Frame #')
plt.ylabel('Median counts/frame')

In [ ]:
#set master flat
flatpath = flat_dir + 'Flat_I__006.fits'
this_flat_stack = bias_subtract(flatpath,biaspath)
master_flat06 = np.average(this_flat_stack, axis=0)
normalized_master_flat06 = master_flat06/np.mean(master_flat06)

## Initalize Fringe Template

In [ ]:
#scale fringe template
#https://danielmuellerkomorowska.com/2020/06/27/contrast-adjustment-with-scikit-image/
from skimage import io, exposure, data

fringe_I = fits.getdata(image_file, ext=0)
fringe_I_minmax_scaled = exposure.rescale_intensity(fringe_I) #scale by minmax
percentiles = np.percentile(fringe_I, (0.5, 99.5)) #scale by percentiles
scaled = exposure.rescale_intensity(fringe_I, in_range=tuple(percentiles)) 
# from PIL import Image, ImageEnhance
# print(type(fringe_I))
# filter = ImageEnhance.Contrast(fringe_I)
# fringe_II = img.filter(float)

plt.imshow(fringe_I, origin='lower', cmap='rainbow', norm=LogNorm())
plt.title('Fringe Template - VATT I ')
plt.colorbar();
plt.show()

plt.imshow(fringe_I_minmax_scaled, origin='lower', cmap='rainbow', norm=LogNorm())
plt.title('Fringe Template - VATT I - min max scaled')
plt.colorbar();
plt.show()

plt.imshow(scaled, origin='lower', cmap='rainbow', norm=LogNorm())
plt.title('Fringe Template - VATT I - precentiles scaled')
plt.colorbar();
plt.show()

# Process Images

In [ ]:
#function that calibrates science stack

def stack_images(filepath,biaspath):
    bias_data = fits.getdata(get_pkg_data_filename(biaspath),ext=0)
    image_stack_raw = fits.getdata(get_pkg_data_filename(filepath),ext=0) 
    image_stack_biased = image_stack_raw - bias_data #subtracts the bias
    image_stack_biased_flatfielded = image_stack_biased / normalized_master_flat06 #divides out the flat field
    image_stack_biased_flatfielded_defringed = image_stack_biased_flatfielded / fringe_I #divides out the fringe
    return(image_stack_biased_flatfielded_defringed) #returns processed image

In [ ]:
#pull specific files for target star and put into list
science_list = os.listdir(science_dir)
science_list = [filename for filename in os.listdir(science_dir) if filename.startswith("LP349")]

def ScienceStackImage(science_FITSfile):
    sciencepath = science_dir + science_FITSfile
    science_stack = stack_images(sciencepath, biaspath)
    science_img = np.sum(science_stack, axis=0)
    return science_img
def ScienceStack(science_FITSfile):
    sciencepath = science_dir + science_FITSfile
    science_stack = stack_images(sciencepath, biaspath)
    return science_stack

science1 = ScienceStackImage('LP349__001.fits')
science2 = ScienceStackImage('LP349__002.fits') 
science3 = ScienceStackImage('LP349__003.fits') 
science4 = ScienceStackImage('LP349__004.fits') 
science5 = ScienceStackImage('LP349__005.fits') 

science_stack1 = ScienceStack('LP349__001.fits')
science_stack2 = ScienceStack('LP349__002.fits') 
science_stack3 = ScienceStack('LP349__003.fits') 
science_stack4 = ScienceStack('LP349__004.fits') 
science_stack5 = ScienceStack('LP349__005.fits') 


def PlotScienceImages(image1, image1title, image2, image2title):
    fig = plt.figure(figsize = (22,6))
    plt.subplot(1, 2, 1)
    plt.imshow(image1, origin='lower', cmap='rainbow', norm = LogNorm())  # use LogNorm to bring out structure
    plt.title(image1title)
    plt.colorbar();
    plt.subplot(1, 2, 2)
    plt.imshow(image2, origin='lower', cmap='rainbow', norm = LogNorm())  # use LogNorm to bring out structure
    plt.title(image2title)
    plt.colorbar();
    return fig

sc12 = PlotScienceImages(science1, 'science1, biased, flat-fielded, defringed, summed', science2, 'science2, biased, flat-fielded, defringed, summed')
sc34 = PlotScienceImages(science3, 'science3, biased, flat-fielded, defringed, summed', science4, 'science4, biased, flat-fielded, defringed, summed')
print(sc12)
print(sc34)


plt.figure(figsize=(8,8))
plt.imshow(science5, origin='lower', cmap='rainbow', norm = LogNorm())  # use LogNorm to bring out structure
plt.title('science5, biased, flat-fielded, defringed, summed')
plt.colorbar();



In [ ]:
#how to search through an image stack for bad data?

science_stack_to_search = science_stack5

for image in science_stack_to_search:
    plt.imshow(image, origin="lower", cmap="rainbow", norm=LogNorm())
    plt.show()
  

# Align Images

In [ ]:
#align individual stacks if needed and then pull out timeseries and normalize then :)


In [ ]:
# import astroalign as aa

# count = 1
# aligned_science4 = []

# for image in science_stack4:
#     source = science_stack4[count]
#     target = science_stack4[0]
#     registered_image, footprint = aa.register(source, target, max_control_points=5)
#     raise Exception (aa.MaxIterError)
#     aligned_science4.append(registered_image)
#     count= count+1


# source = science_stack4[30]
# target = science_stack4[0]
# registered_image, footprint = aa.register(source, target, max_control_points=5)

In [ ]:
# offset_image = science_stack4[30]
# image = science_stack4[0]

# from image_registration import chi2_shift
# from image_registration.fft_tools import shift
# xoff, yoff, exoff, eyoff = chi2_shift(image, offset_image,  return_error=True, upsample_factor='auto')
# corrected_image2 = shift.shiftnd(offset_image, (-yoff, -xoff))

## Deeper Look into Science Frames

In [ ]:
#Use sep module to see deeper 

sc_images = [science1, science2, science3, science4, science5]
nums = [1,2,3,4,5]
def SEPPlot(science_images, title):
    m, s = np.mean(science_images), np.std(science_images)
    fig = plt.figure(figsize=(8,8))
    plt.imshow(science_images, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
    plt.title(title)
    plt.colorbar();
    plt.show()
    return fig

count = 1
for i in sc_images:
    plots = SEPPlot(i, count)
    count = count+1
    print(plots)
    


## Visualize fringe removal

In [ ]:
#comparing fringe with defringe

def stack_images_test(filepath,biaspath):
    bias_data = fits.getdata(get_pkg_data_filename(biaspath),ext=0)
    image_stack_raw = fits.getdata(get_pkg_data_filename(filepath),ext=0)
    image_stack_biased = image_stack_raw - bias_data
    image_stack_biased_flatfielded = image_stack_biased / normalized_master_flat06
    return(image_stack_biased_flatfielded)

sciencepath_test = science_dir + 'LP349__002.fits'
this_science_stack_test = stack_images_test(sciencepath_test,biaspath)
science_test = np.sum(this_science_stack_test, axis=0)
# m, s = np.mean(science_test), np.std(science_test)
# plt.figure(figsize=(8,8))
# plt.imshow(science_test, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
# plt.show()

#plot fringe comparison
plt.figure(figsize = (22,6))
plt.subplot(1, 2, 1)
plt.title('science 2 fringing removed')
m, s = np.mean(science2), np.std(science2)
plt.imshow(science2, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

plt.subplot(1, 2, 2)
plt.title('science 2 w/o fringing removed')
m, s = np.mean(science_test), np.std(science_test)
plt.imshow(science_test, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

# Determine Sources and Plot Time Series

Defining functions to do heavy lifting

In [ ]:
def FindObjects(science_image):
    
    # measure a spatially varying background on the image
    bkg = sep.Background(science_image)
    # get a "global" mean and noise of the image background:
    # evaluate background as 2-d array, same size as original image
    bkg_image = bkg.back()
    # evaluate the background noise as 2-d array, same size as original image
    bkg_rms = bkg.rms()
    # look for flat-field artifacts and possible vignetting
    # subtract the background
    science_sub = science_image - bkg

    #show the background
    bkgplot = plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
    plt.title('show background')
    plt.colorbar();
    plt.show()
    #show the background noise
    bkgnoiseplot = plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
    plt.title('show background noise')
    plt.colorbar();
    plt.show()
    #show the background subtracted image
    m, s = np.mean(science_sub), np.std(science_sub)
    bkgremplot = plt.imshow(science_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
    plt.title('bkg sub image')
    plt.colorbar();
    plt.show()

    #SEP search with sigma=15 in field
    objects = sep.extract(science_sub, 20 , err=bkg.globalrms)
    #incorporate sections with summed images
    from matplotlib.patches import Ellipse

    # plot image with bkg sub 
    fig, ax = plt.subplots()
    m, s = np.mean(science_sub), np.std(science_sub)
    im = ax.imshow(science_sub, interpolation='nearest', cmap='gray',
                   vmin=m-s, vmax=m+s, origin='lower')

    # plot an ellipse for each object
    for i in range(len(objects)):
        e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                    width=6*objects['a'][i],
                    height=6*objects['b'][i],
                    angle=objects['theta'][i] * 180. / np.pi)
        e.set_facecolor('none')
        e.set_edgecolor('red')
        ax.add_artist(e)

    #use SEP to determine flux, flux error, on each object
    flux, fluxerr, flag = sep.sum_circle(science_sub, objects['x'], objects['y'],7.0, err=bkg.globalrms, gain=1.0)
    for i in range(len(objects)):
        print("object {:d}: Flux= {:f} +/- {:f} --- Position= {:f},{:f}".format(i, flux[i], fluxerr[i], objects[i][7],objects[i][8]))

    return bkgplot, bkgnoiseplot, bkgremplot, fig, ax

In [ ]:
def PlotTimeseries(science_image, science_stack, title, target, r1, r2, r3, r4, r5, ):


    
#     ref_range = [0,1,2,3,4,5]
#     for i in ref_range:
#         if i == target:
#             none
#         else:
            
    
    bkg = sep.Background(science_image)
    bkg_image = bkg.back()
    bkg_rms = bkg.rms()
    science_sub = science_image - bkg
    objects = sep.extract(science_sub, 20 , err=bkg.globalrms)



#     # plot an ellipse for each object
#     for i in range(len(objects)):
#         e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
#                     width=6*objects['a'][i],
#                     height=6*objects['b'][i],
#                     angle=objects['theta'][i] * 180. / np.pi)
#         e.set_facecolor('none')
#         e.set_edgecolor('red')
#         ax.add_artist(e)

#     #use SEP to determine flux, flux error, on each object
#     flux, fluxerr, flag = sep.sum_circle(science_sub, objects['x'], objects['y'],7.0, err=bkg.globalrms, gain=1.0)

    
    
    target_flx = []
    target_errflx = []
    ref1_flx = []
    ref1_errflx = []
    ref2_flx = []
    ref2_errflx = []
    ref3_flx = []
    ref3_errflx = []
    ref4_flx = []
    ref4_errflx = []
    ref5_flx = []
    ref5_errflx = []
    frame_no = 0
    
    for frame in science_stack:
        #measure spatially varying bkg on each frame
        bkg = sep.Background(frame)
        #evaluate bkg as 2d array
        bkg_image = bkg.back()
        #evaluate kg noise as 2d array
        bkg_rms = bkg.rms()
        #subtract the bkg
        frame_sub = frame - bkg
        #circular apertture photometry with 7 pixel radius at locations of the objects!
        flux, fluxerr, flag = sep.sum_circle(frame_sub, objects['x'], objects['y'], 7.0, err= bkg.globalrms, gain=1.0)
        target_flx.append(flux[target])
        target_errflx.append(fluxerr[target])
        ref1_flx.append(flux[r1])
        ref1_errflx.append(fluxerr[r1])
        ref2_flx.append(flux[r2])
        ref2_errflx.append(fluxerr[r2])
        ref3_flx.append(flux[r3])
        ref3_errflx.append(fluxerr[r3])
        ref4_flx.append(flux[r4])
        ref4_errflx.append(fluxerr[r4])
        ref5_flx.append(flux[r5])
        ref5_errflx.append(fluxerr[r5])
        frame_no = frame_no + 1

    #sum lists element wise using list comprehension, to add up the ref stars 
    ref_stars = [sum(x) for x in zip(ref1_flx, ref2_flx, ref3_flx, ref4_flx, ref5_flx )]
    ref_stars_err = [sum(x) for x in zip(ref1_errflx, ref2_errflx, ref3_errflx, ref4_errflx, ref5_errflx )]


    #Plot all stars
    import matplotlib.colors as mcolors
    frame_no = range(0,len(target_flx))
    fig=plt.figure(figsize=(16, 6))
    plt.plot(frame_no,target_flx, 'coral'); #lp349
    plt.plot(frame_no,ref1_flx, "black"); #ref
    plt.plot(frame_no,ref2_flx, "dimgrey"); #ref
    plt.plot(frame_no,ref3_flx, "grey"); #ref
    plt.plot(frame_no,ref4_flx, "silver"); #ref
    plt.plot(frame_no,ref5_flx, "lightgrey"); #ref
    plt.title('{} target and ref stars '.format(title))
    plt.xlabel('Frame Number')
    plt.ylabel('Relative flux ')
    plt.show()

    #Plot target star and summed ref stars
    frame_no = range(0,len(target_flx))
    fig=plt.figure(figsize=(16, 6))
    plt.plot(frame_no,target_flx, 'coral'); #lp349
    plt.plot(frame_no,ref_stars, "mediumpurple"); #ref
    plt.title('{} target (orange) and summed ref stars (purple)'.format(title))
    plt.xlabel('Frame Number')
    plt.ylabel('Relative flux ')
    plt.show()

    #Plot Differential 
    target_flux = np.array(target_flx)
    ref_stars = np.array(ref_stars)
    dflux = -2.5*np.log10(ref_stars/target_flux)

    target_flux_all_err = np.array(target_errflx)
    ref_stars_all_err = np.array(ref_stars_err)
    dflux_err = -2.5*np.log10(ref_stars_all_err/target_flux_all_err)

    frame_no = range(0,len(target_flx))
    fig=plt.figure(figsize=(16, 6))
    plt.plot(frame_no, dflux, 'dodgerblue');
    plt.title('{} Differential Photometry for lp349 and summed ref stars'.format(title))
    plt.xlabel('Frame Number')
    plt.ylabel('Relative Flux Difference')
    plt.show();

    return dflux, dflux_err, ref_stars, ref_stars_all_err

In [ ]:
def PlotTimeseries_lessref(science_image, science_stack, title, target, r1, r2, r3, r4):


    
#     ref_range = [0,1,2,3,4,5]
#     for i in ref_range:
#         if i == target:
#             none
#         else:
            
    
    bkg = sep.Background(science_image)
    bkg_image = bkg.back()
    bkg_rms = bkg.rms()
    science_sub = science_image - bkg
    objects = sep.extract(science_sub, 20 , err=bkg.globalrms)



#     # plot an ellipse for each object
#     for i in range(len(objects)):
#         e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
#                     width=6*objects['a'][i],
#                     height=6*objects['b'][i],
#                     angle=objects['theta'][i] * 180. / np.pi)
#         e.set_facecolor('none')
#         e.set_edgecolor('red')
#         ax.add_artist(e)

#     #use SEP to determine flux, flux error, on each object
#     flux, fluxerr, flag = sep.sum_circle(science_sub, objects['x'], objects['y'],7.0, err=bkg.globalrms, gain=1.0)

    
    
    target_flx = []
    target_errflx = []
    ref1_flx = []
    ref1_errflx = []
    ref2_flx = []
    ref2_errflx = []
    ref3_flx = []
    ref3_errflx = []
    ref4_flx = []
    ref4_errflx = []
    ref5_flx = []
    ref5_errflx = []
    frame_no = 0
    
    for frame in science_stack:
        #measure spatially varying bkg on each frame
        bkg = sep.Background(frame)
        #evaluate bkg as 2d array
        bkg_image = bkg.back()
        #evaluate kg noise as 2d array
        bkg_rms = bkg.rms()
        #subtract the bkg
        frame_sub = frame - bkg
        #circular apertture photometry with 7 pixel radius at locations of the objects!
        flux, fluxerr, flag = sep.sum_circle(frame_sub, objects['x'], objects['y'], 7.0, err= bkg.globalrms, gain=1.0)
        target_flx.append(flux[target])
        target_errflx.append(fluxerr[target])
        ref1_flx.append(flux[r1])
        ref1_errflx.append(fluxerr[r1])
        ref2_flx.append(flux[r2])
        ref2_errflx.append(fluxerr[r2])
        ref3_flx.append(flux[r3])
        ref3_errflx.append(fluxerr[r3])
        ref4_flx.append(flux[r4])
        ref4_errflx.append(fluxerr[r4])
      
        frame_no = frame_no + 1

    #sum lists element wise using list comprehension, to add up the ref stars 
    ref_stars = [sum(x) for x in zip(ref1_flx, ref2_flx, ref3_flx, ref4_flx )]
    ref_stars_err = [sum(x) for x in zip(ref1_errflx, ref2_errflx, ref3_errflx, ref4_errflx)]


    #Plot all stars
    import matplotlib.colors as mcolors
    frame_no = range(0,len(target_flx))
    fig=plt.figure(figsize=(16, 6))
    plt.plot(frame_no,target_flx, 'coral'); #lp349
    plt.plot(frame_no,ref1_flx, "black"); #ref
    plt.plot(frame_no,ref2_flx, "dimgrey"); #ref
    plt.plot(frame_no,ref3_flx, "grey"); #ref
    plt.plot(frame_no,ref4_flx, "silver"); #ref
    #plt.plot(frame_no,ref5_flx, "lightgrey"); #ref
    plt.title('{} target and ref stars '.format(title))
    plt.xlabel('Frame Number')
    plt.ylabel('Relative flux ')
    plt.show()

    #Plot target star and summed ref stars
    frame_no = range(0,len(target_flx))
    fig=plt.figure(figsize=(16, 6))
    plt.plot(frame_no,target_flx, 'coral'); #lp349
    plt.plot(frame_no,ref_stars, "mediumpurple"); #ref
    plt.title('{} target (orange) and summed ref stars (purple)'.format(title))
    plt.xlabel('Frame Number')
    plt.ylabel('Relative flux ')
    plt.show()

    #Plot Differential 
    target_flux = np.array(target_flx)
    ref_stars = np.array(ref_stars)
    dflux = -2.5*np.log10(ref_stars/target_flux)

    target_flux_all_err = np.array(target_errflx)
    ref_stars_all_err = np.array(ref_stars_err)
    dflux_err = -2.5*np.log10(ref_stars_all_err/target_flux_all_err)

    frame_no = range(0,len(target_flx))
    fig=plt.figure(figsize=(16, 6))
    plt.plot(frame_no, dflux, 'dodgerblue');
    plt.title('{} Differential Photometry for lp349 and summed ref stars'.format(title))
    plt.xlabel('Frame Number')
    plt.ylabel('Relative Flux Difference')
    plt.show();

    return dflux, dflux_err, ref_stars, ref_stars_all_err

Apply and show results

In [ ]:
science_objects1 = FindObjects(science1)
science_timeseries1 = PlotTimeseries(science1, science_stack1, 'Science 1 (21/12/17)' ,3, 0, 1, 2, 4, 5)

In [ ]:
science_objects2 = FindObjects(science2)
science_timeseries2 = PlotTimeseries_lessref(science2, science_stack2, 'Science 2 (21/12/17)' , 2, 0, 1, 3, 4 )

In [ ]:
science_objects3 = FindObjects(science3)
science_timeseries3 = PlotTimeseries_lessref(science3, science_stack3, 'Science 3 (21/12/17)' , 2, 0, 1, 3, 4, )

In [ ]:
science_objects4 = FindObjects(science4)
science_timeseries4 = PlotTimeseries(science4, science_stack4, 'Science 4 (21/12/17)' , 3, 0, 1, 2, 4, 5)

In [ ]:
# science_objects5 = FindObjects(science5)
# science_timeseries5 = PlotTimeseries(science5, science_stack5, 'Science 5 (21/12/17)' , 2, 0, 1, 3, 4, 5)

## initalize differential fluxes

In [ ]:
# differential fluxes
dflux1 = (science_timeseries1[0])
dflux2 = (science_timeseries2[0])
dflux3 = (science_timeseries3[0])
dflux4 = (science_timeseries4[0])

dflux1_err = (science_timeseries1[1])
dflux2_err = (science_timeseries2[1])
dflux3_err = (science_timeseries3[1])
dflux4_err = (science_timeseries4[1])

count = 0
flux_list = [dflux1, dflux2, dflux3, dflux4]
flux_err_list = [dflux1_err, dflux2_err, dflux3_err, dflux4_err]
flux_names = ['dflux1', 'dflux2', 'dflux3', 'dflux4']
flux_err_names = ['dflux1_err', 'dflux2_err', 'dflux3_err', 'dflux4_err']

#convert to arrays and check shape
for i in flux_list:
    i = np.array(i)
    print(flux_names[count],'shape:' ,i.shape)
for k in flux_err_list:
    k = np.array(k)
    print(flux_err_names[count],'shape:' ,k.shape)
    count = count + 1
   

# Time Stamp

In [ ]:
#change directory to science fits files
#then create list of initial datetime for every fits file in directory
os.chdir("/Volumes/KT/data/2017_12_21/science/LP349")
init_times = []
for i in science_list:
    hdul = fits.open(i)
    init_time = hdul[0].header["DATE"]
    init_times.append(init_time)
print('Inital Times per FITS:',(init_times))

#create a iterated timeseries given start and time difference
def Timeseriesobject(inital_time, delta_time_sec, samples, flux):
    ts = TimeSeries(time_start = inital_time, time_delta = delta_time_sec * u.s, n_samples= samples, masked=True)
    ts['flux'] = flux
    return ts

ts1 = Timeseriesobject(init_times[0], 5, 600, dflux1)
ts2 = Timeseriesobject(init_times[1], 5, 81, dflux2)
ts3 = Timeseriesobject(init_times[2], 5, 600, dflux3)
ts4 = Timeseriesobject(init_times[3], 5, 600, dflux4)

ts1_err = Timeseriesobject(init_times[0], 5, 600, dflux1_err)
ts2_err = Timeseriesobject(init_times[1], 5, 81, dflux2_err)
ts3_err = Timeseriesobject(init_times[2], 5, 600, dflux3_err)
ts4_err = Timeseriesobject(init_times[3], 5, 600, dflux4_err)

from astropy.table import Table, hstack, vstack
ts_tot = vstack([ts1, ts2, ts3, ts4])
ts_tot_err = vstack([ts1_err, ts2_err, ts3_err, ts4_err])

ts_list = [ts1, ts2, ts3, ts4]
ts_err_list = [ts1_err, ts2_err, ts3_err, ts4_err]

#check what time scale is the time on?
print('Time Scale:', ts1.time.scale)
#convert to mjd or jd
#ts1.time.jd

## Plot Time-Stamped Differential Data

In [ ]:
#plot timeseries
fig=plt.figure(figsize=(16, 6))
plt.plot(ts_tot.time.jd, ts_tot['flux'], 'k.', markersize =1)
plt.title('Differential Flux from all 21/12/17 Observations')
plt.xlabel('Julian Date')
plt.ylabel('Flux')
plt.show()

## Normalize Differential Data

In [ ]:
#Normalize Differential Flux
for data in ts_list:
    mean, median, stddev = sigma_clipped_stats(data['flux'])
    data['flux_norm'] = data['flux'] / mean
ts_tot = vstack([ts1, ts2, ts3, ts4])

#Normalize Differential Flux Error
for data_err in ts_err_list:
    mean, median, stddev = sigma_clipped_stats(data_err['flux'])
    data_err['flux_norm'] = data_err['flux'] / mean
    
#Re-Stack the Normalized Segments    
ts_tot = vstack([ts1, ts2, ts3, ts4])
ts_tot_err = vstack([ts1_err, ts2_err, ts3_err, ts4_err])

#Plot
fig=plt.figure(figsize=(16, 6))
plt.plot(ts_tot.time.jd, ts_tot['flux_norm'], 'k.', markersize=1)
plt.title('Normalized Differential Flux 21/12/17')
plt.xlabel('Time JD')
plt.ylabel('Normalized flux')

## Clip Erroneous Data

In [ ]:
#From astropy - clip data that falls outside 3 standard deiviations 
sigclip = SigmaClip(sigma=3, cenfunc='mean',  maxiters =10)
ts_tot_clip = sigclip(ts_tot['flux_norm'])
ts_tot_err_clip = sigclip(ts_tot_err['flux_norm'])
ts_tot_clip

#Plot
fig=plt.figure(figsize=(16, 6))
plt.plot(ts_tot.time.jd, ts_tot_clip, 'k', markersize =1)
plt.title('Sigma clipped - LP349 21/12/17')
plt.xlabel('Julian Date')
plt.ylabel('Flux')
plt.ylim(0, 2)
plt.show()

## Lombscargle 

In [ ]:
#convert times relative to 0 
t_rel = TimeSeries(time = ts_tot.time - ts_tot.time[0])

#define times and values for periodogram
times = t_rel.time.to('hour')
values = ts_tot_clip
frequency, power = LombScargle(times, values).autopower()

#Print off some stats
print('Frequency Units:', frequency.unit)
print('Power Units:', power.unit)
max_power = frequency[np.argmax(power)]
print("Max Power:", max_power)

#Plot
plt.plot(frequency, power)
plt.title('Lombscargle Periodogram LP349 21/12/17')
plt.xlabel('Frequency 1/H')
plt.ylabel('Power')
plt.show()

## Phase Folding

In [ ]:
#https://pyastronomy.readthedocs.io/en/latest/pyaslDoc/aslDoc/folding.html

# Obtain the phases with respect to some times
phases = foldAt(times, 1/max_power , T0=0)

# Sort according to phase
#get the order of indices
sortIndi = np.argsort(phases)
#rearrange the arrays.
phases = phases[sortIndi]
flux = ts_tot_clip[sortIndi]

# Plot
fig=plt.figure(figsize=(16, 6))
plt.plot(phases, flux, 'k')
plt.title('Phase Folded LP349 21/12/17')
plt.xlabel('Phases')
plt.ylabel('Flux')
plt.show()